In [16]:
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
import subprocess as sp

In [20]:
from arboreto.algo import grnboost2
from arboreto.utils import load_tf_names
import scanpy as sc

ImportError: Numba needs NumPy 2.0 or less. Got NumPy 2.2.

In [ ]:
def fetch_adata(adata):
    return csc_matrix(adata.X).toarray(), adata.var_names.values, adata.obs_names.values

In [ ]:
adata = sc.read_h5ad(data_dir)
tf_names = load_tf_names(tf_dir)

In [ ]:
data_dir = "/home/igarzonalva/Proyecto_SC_TNBC/GSE161529/02_Integration/adata/adata_scanvi_cuda_refinement.h5ad"
tf_dir = "/home/igarzonalva/Proyecto_SC_TNBC/GSE161529/04_grn/Common_files/TF_names_v_1.01.txt"
network_dir = "/home/igarzonalva/Proyecto_SC_TNBC/GSE161529/04_grn/01Networks"

In [ ]:
for tumor_type in ['ER', 'HER2', 'TNBC']:
    adata_sub = adata[adata.obs['subtype'] == tumor_type,:].copy()
    mat, genes, cells = fetch_adata(adata_sub)
    n_genes = len(genes)
    gene_names = pd.Series(genes)
    n_matching_genes = gene_names.isin(tf_names).sum() # cuántos de los genes de la matriz de expresión son TFs
    print(type(mat))
    print(mat.shape)
    print(f'the number of genes is {n_genes}')
    print(f'Out of {n_genes} genes, {n_matching_genes} are TFs')
    portdash = 40748
    hostname = socket.gethostname()
    cluster = SLURMCluster(queue = "short", cores=16, processes=1, 
                       memory="16GB", walltime="05:00:00",
                       scheduler_options={"dashboard_address": f":{portdash}", "host": 'nodo05'})
    cluster.scale(6)
    client = Client(cluster)
    #mat_future = client.scatter(mat, broadcast=True)

    print(client)
    print(cluster)
    print(client.scheduler_info())
    network = grnboost2(
                expression_data=mat,
                gene_names=genes,
                tf_names=tf_names,
                client_or_address=client)
    network_file = os.path.join(network_dir, f"{tumor_type}_network.tsv")
    network.to_csv(network_file, sep='\t', header=False, index=False)
    
    client.close()
    cluster.close()